In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 36.8 MB/s 
     |████████████████████████████████| 61 kB 475 kB/s 
     |████████████████████████████████| 596 kB 75.4 MB/s 
     |████████████████████████████████| 895 kB 63.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [86]:
import pandas as pd
import numpy as np
import transformers
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
import json
import tqdm

In [3]:
phobert = AutoModel.from_pretrained("vinai/phobert-base",output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#Demo

In [29]:
line = "Tôi là sinh_viên trường đại_học Công_nghệ ."

token = tokenizer.encode_plus(line,line)

In [30]:
token

{'input_ids': [0, 218, 8, 649, 212, 956, 2413, 5, 2, 2, 218, 8, 649, 212, 956, 2413, 5, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [33]:
with torch.no_grad():
    features = phobert(input_ids = torch.tensor([token['input_ids']]))  # Models outputs are now tuples

In [38]:
features['last_hidden_state'].shape

torch.Size([1, 18, 768])

In [ ]:
tokenizer.convert_ids_to_tokens(token['input_ids'])

In [82]:
token['input_ids']

[0, 218, 8, 649, 212, 956, 2413, 5, 2, 2, 218, 8, 649, 212, 956, 2413, 5, 2]

In [45]:
token['input_ids'][0:token['input_ids'].index(2)+1]

[0, 218, 8, 649, 212, 956, 2413, 5, 2]

In [83]:
temp = torch.tensor([[[0, 218, 8, 649, 212, 956, 2413, 5, 2, 2, 218, 8, 649, 212, 956, 2413, 5, 2],
                     [0, 218, 8, 649, 212, 956, 2413, 5, 2, 2, 218, 8, 649, 212, 956, 2413, 5, 2],
                     [0, 218, 8, 649, 212, 956, 2413, 5, 2, 2, 218, 8, 649, 212, 956, 2413, 5, 2]],
                     [[0, 218, 8, 649, 212, 956, 2413, 5, 2, 2, 218, 8, 649, 212, 956, 2413, 5, 2],
                     [0, 218, 8, 649, 212, 956, 2413, 5, 2, 2, 218, 8, 649, 212, 956, 2413, 5, 2],
                     [0, 218, 8, 649, 212, 956, 2413, 5, 2, 2, 218, 8, 649, 212, 956, 2413, 5, 2]]])

In [84]:
temp.shape

torch.Size([2, 3, 18])

In [63]:
temp_list = temp.tolist()

In [64]:
result = []
for i in temp_list:
  temp = []
  for j in i:
    index = j.index(2)
    temp.append(j[:index+1])
  result.append(temp)

In [14]:
features.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])

#Train

In [90]:
mlqadata_df = pd.read_json('/content/drive/MyDrive/Fine-Tune-Bert/data/mlqa_vi_zalo_format.json')
xquaddata_df = pd.read_json('/content/drive/MyDrive/Fine-Tune-Bert/data/xquad_vi_zaloformat_training_data.json')
zalodata_df = pd.read_json('/content/drive/MyDrive/Fine-Tune-Bert/data/trainingdata_zalo.json')

In [91]:
zalodata_df = zalodata_df.rename({'text':'context'},axis =1)
trainingdata = pd.concat((zalodata_df,mlqadata_df,xquaddata_df))

In [92]:
cuda = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [93]:
columns = ["context", 'question','label']
data = []
for index, row in trainingdata.iterrows():
  temp = []
  temp.append(row['context'])
  temp.append(row['question'])
  temp.append(int(row['label']))
  data.append(temp)

In [94]:
phobert = AutoModel.from_pretrained("vinai/phobert-base",output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [95]:
data = pd.DataFrame(data, columns=columns)
data = data.sample(frac = 1)

In [87]:
class TextDataset(Dataset):
  def __init__(self, text, question, label, tokenizer, max_length):
    super(TextDataset, self).__init__()
    self.tokenizer = tokenizer
    self.text = text
    self.question = question
    self.label = label
    self.max_length = max_length
  def __len__(self):
    return len(self.label)
  def __getitem__(self, index):
    inputs = self.tokenizer.encode_plus(
        self.question[index],self.text[index],
        pad_to_max_length = True,
        add_special_tokens = True,
        return_attention_mask = True,
        max_length = self.max_length,
        truncation = True,
    )
    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]

    return{
        "ids" : torch.tensor(ids, dtype = torch.long, device = cuda),
        "attention_mask" : torch.tensor(mask, dtype = torch.long, device=cuda),
        "target" : torch.tensor(self.label[index], dtype = torch.long, device=cuda),
    }

In [96]:
dataset = TextDataset(data['context'],data['question'],data['label'],tokenizer, 256)
datatrainloader = DataLoader(dataset=dataset, batch_size=16)

In [105]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(LSTM, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size

    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
  def forward(self, input):
    output,_ = self.lstm(input)
    return output

In [122]:
ids.shape

torch.Size([16, 256])

In [130]:
print(ids)

tensor([[    0, 11598,  3078,  ...,     1,     1,     1],
        [    0, 11598,  3078,  ...,     1,     1,     1],
        [    0,  1597,  2654,  ...,     1,     1,     1],
        ...,
        [    0,   319,   964,  ...,     1,     1,     1],
        [    0,   319,   504,  ...,     1,     1,     1],
        [    0,  3333,  4138,  ...,     1,     1,     1]])


In [131]:
temp_list = ids.tolist()
result = []
for i in temp_list:
  temp = []
  temp.append(i[:i.index(2)+1] +[0]*(30-i.index(2)-1))
  result.append(temp)

In [138]:
ids[0].tolist().index(2)

88

In [136]:
print(np.array(result).shape)

(16, 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [116]:
def getQuestionVector(input, max_question_length):
  temp_list = input.tolist()
  result = []
  for i in temp_list:
    temp.append(i[:i.index(2)+1] +[0]*(max_question_length-i.index(2)-1))
    result.append(temp)
  return torch.tensor(result)

In [81]:
def getTextVector(input, max_text_length, output):
  temp_list = input.tolist()
  result = []
  for i in temp_list:
    temp = []
    for j in i:
      temp.append(j[j.index(2)+1:] +[0]*(max_text_length-len(j[j.index(2)+1:])))
    result.append(temp)

  return torch.tensor(result)

In [106]:
class BERT(nn.Module):
  def __init__(self, bert):
    super(BERT, self).__init__()
    self.bert = bert
    self.linear_1 = nn.Linear(768*4, 2)
    self.lstm_question = LSTM(input_size=768, hidden_size= 30)
    self.lstm_context = LSTM(input_size=768, hidden_size= 256)
    
  def forward(self, input_ids, attention_mask):
    output = self.bert(input_ids = input_ids, attention_mask = attention_mask)
    print(output['last_hidden_state'].shape)
    cls_output = torch.cat((output['hidden_states'][-1][:,0,:],
                            output['hidden_states'][-2][:,0,:],
                            output['hidden_states'][-3][:,0,:],
                            output['hidden_states'][-4][:,0,:]), dim = -1)
    question = self.lstm_question(output['last_hidden_state'])

    output = self.linear_1(cls_output)
    output = self.linear_2(output)
    # output = self.sigmoid(output)
    return output

In [107]:
model = BERT(phobert).to(cuda)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-6)

In [109]:
for param in model.bert.parameters():
  param.requires_grad = False
transformers.logging.set_verbosity_error()

In [120]:
ids[0].shape

torch.Size([256])

In [117]:
getQuestionVector(ids,64)

ValueError: ignored

In [114]:
ids.shape

torch.Size([16, 256])

In [112]:
model.train()
for epoch in range(5):
  print(epoch)
  loop = tqdm.tqdm(enumerate(datatrainloader), leave=False, total=len(datatrainloader))
  for batch, dl in loop:
    ids = dl['ids']
    attention_mask = dl['attention_mask']
    label = dl['target']
    print(ids.shape)

    optimizer.zero_grad()
    output = model(input_ids=ids, attention_mask = attention_mask)
    # print(output)
    # print(label)
    # temp = torch.tensor(one_hot(label, 2),dtype=torch.float, device=cuda)
    loss = loss_function(output, label)

    predict =torch.argmax(output, dim = 1)
    correct = sum(1 for a, b in zip(predict, label) if a == b)
    samples = output.shape[0]
    accuracy = correct/samples
    loss.backward()
    optimizer.step()

    loop.set_description(f'Epoch={epoch}/10')
    loop.set_postfix(loss=loss.item(),acc=accuracy)

0


  0%|          | 0/992 [00:00<?, ?it/s]

torch.Size([16, 256])


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


KeyboardInterrupt: ignored